## Smali & Manifest File Analysis

This notebook will try to analyze a directory and its SMALI and MANIFEST files. The analysis will find and print sensitive methods and packages used by the program. 

In [2]:
from os import walk
import pandas as pd

## Initially, we declare the lists, and read csvs into the dataframe.

In [3]:
methods_to_find = []
perms_to_find = []
found_perms = []
found_lines = []

#read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions
df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [4]:
class obj:
    def __init__(self, name):
        self.name = name
        self.count = 0
    def print(self):
        print(self.name + ": " + str(self.count))

In [5]:
myList = []
fakeList = []

for i in perms_to_find:
    myList.append(obj(i))
    fakeList.append(i)

for i in methods_to_find:
    myList.append(obj(i))
    fakeList.append(i)

def printList():
    for i in range(len(myList)):
        myList[i].print()

In [6]:
def findFunctionsInDir(dir, list=0, verbose = True):
    #for each directory, and files
    for dirpath, dirnames, files in walk(dir):

        #skip the res folder, too many subfolders, slows.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):
                                #print the harmful line
                                
                                myList[fakeList.index(i)].count += 1
                                if(line.strip() not in found_lines):
                                    if(verbose == True): print(num, ": ", line)
                                    found_lines.append(line.strip())

            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                
                                #print the harmful line
                                myList[fakeList.index(i)].count += 1
                                
                                if(line.strip() not in found_perms):
                                    if(verbose == True): print(num, ": ", line)
                                    found_perms.append(line.strip())

In [7]:
directory = "C:\\Users\\Vin\\Desktop\\apktool\\Sample1-WannaLocker\\sample1"

findFunctionsInDir(dir=directory, verbose=False)
printList()

READ_CALENDAR: 0
WRITE_CALENDAR: 0
CAMERA: 0
READ_CONTACTS: 0
WRITE_CONTACTS: 0
GET_ACCOUNTS: 0
ACCESS_FINE_LOCATION: 0
ACCESS_COARSE_LOCATION: 0
RECORD_AUDIO: 0
READ_PHONE_STATE: 1
READ_PHONE_NUMBERS: 0
CALL_PHONE: 0
ANSWER_PHONE_CALLS: 0
READ_CALL_LOG: 0
WRITE_CALL_LOG: 0
ADD_VOICEMAIL: 0
USE_SIP: 0
PROCESS_OUTGOING_CALLS: 0
BODY_SENSORS: 0
SEND_SMS: 0
RECEIVE_SMS: 0
READ_SMS: 0
RECEIVE_WAP_PUSH: 0
RECEIVE_MMS: 0
READ_EXTERNAL_STORAGE: 1
WRITE_EXTERNAL_STORAGE: 1
CALL_LOG: 0
DEVICE_ADMIN_ENABLED: 0
SET_WALLPAPER: 1
USER_PRESENT: 0
QUICKBOOT_POWERON: 0
BOOT_COMPLETED: 0
REBOOT: 0
SYSTEM_ALERT_WINDOW: 0
GET_TASKS: 1
MOUNT_UNMOUNT_FILESYSTEMS: 1
sendTextMessage: 0
sendDataMessage: 0
sendMultimediaMessage: 0
sendMultipartTextMessage: 0
setReadOnly: 0
setReadable: 0
setWritable: 0
Cipher: 22
execSQL: 0
SQLiteDatabase: 0
bindSocket: 0
openConnection: 1
getFilesDir: 1
getCacheDir: 1
getExternalFilesDir: 4
getExternalCacheDir: 4
shareIntent: 0
readPhoneNumber: 0
FileInputStream: 19
getShared

In [8]:
def saveToFile():
    file1 = open("output2.csv","a")
    for i in range(len(myList)):
        file1.write(myList[i].name + "," + str(myList[i].count) + ",\n")
    file1.close()

In [9]:
saveToFile()

In [10]:
df = pd.DataFrame(columns=fakeList)
df

,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,getExternalCacheDir,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto


In [22]:
listOfCount = []
for i in range(len(myList)):
    listOfCount.append(myList[i].count)

df.loc[len(df.index)] = listOfCount
df

C:\Users\Vin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,getExternalCacheDir,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto
0,0,0,0,0,0,0,0,0,0,1,...,4,0,0,19,8,0,0,0,0,37


In [19]:
df = df.iloc[0:0]